This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [58]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#59.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [59]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudyBugFix");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 8; Grid Count = 8; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudyBugFix }

In [60]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-31 21:27:03Z

In [61]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:22:09 PM	f868f5b3...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:21:15 PM	704ed890...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/31/2024 9:40:07 PM	00bf8f53...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/31/2024 9:36:02 PM	629de94e...
#4: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/31/2024 9:30:21 PM	8527b4fd...
#5: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:18:57 PM	185058d1...
#6: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0*	3/31/2024 9:48:09 PM	38f93025...
#7: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	3/31/2024 9:27:03 PM	625022d8...


In [62]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

f868f5b3-03c2-499e-a739-35f200b766b8

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [63]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [64]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [65]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("iFlx0") &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:22:09 PM	f868f5b3...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:21:15 PM	704ed890...
#2: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:18:57 PM	185058d1...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	3/31/2024 9:27:03 PM	625022d8...


# A Helper Function to obtain different plot formats

In [66]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [67]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M500.1,57.1 L553.5,57.1 M45.6,347.2 L51.1,348.0 L56.6,348.5 L62.2,349.3 L67.7,352.4 L73.2,356.7
 L78.7,358.1 L84.3,358.7 L89.8,359.3 L95.3,361.5 L100.8,364.3 L106.3,365.1 L111.9,365.4 L117.4,365.4
 L122.9,365.4 L128.4,365.4 L134.0,365.4 L139.5,365.4 L145.0,365.4 L150.5,365.4 L156.1,365.4 L161.6,365.4
 L167.1,365.4 L172.6,365.4 L178.1,365.4 L183.7,365.4 L189.2,365.4 L194.7,365.4 L200.2,365.4 L205.8,365.4
 L211.3,365.4 L216.8,365.4 L222.3,365.4 L227.8,365.4 L233.4,365.4 L238.9,365.4 L244.4,365.4 L249.9,365.4
 L255.5,365.4 L261.0,365.4 L266.5,365.4 L272.0,365.4 L277.5,365.4 L283.1,365.4 L288.6,365.4 L294.1,365.4
 L299.6,365.4 L305.2,365.4 L310.7,365.4 L316.2,365.4 L321.7,365.4 L327.2,371.6 L332.8,376.7 L338.3,382.7
 L343.8,385.8 L349.3,390.4 L354.9,401.8 L360.4,408.9 L365.9,418.7 L371.4,425.0 L377.0,434.9 L382.5,435.4
 L388.0,435.8 L393.5,435.7 L399.0,435.7 L404.6,435.7 L410.1,435.7 L415.6,435.7 L421.1,435.7 L426.7,435.7
 L432.2,435.7 L437.7,435.7 L443.2,435.7 L448.7,435.7 L454.3,435.7 L459.8,435.7 L465.3,435.7 L470.8,435.7
 L476.4,435.7 L481.9,435.7 L487.4,435.7 L492.9,435.7 L498.4,435.7 L504.0,435.7 L509.5,435.7 L515.0,435.7
 L520.5,435.7 L526.1,435.7 L531.6,435.7 L537.1,435.7 L542.6,435.7 L548.1,435.7 L553.7,435.7 L559.2,435.7
 L564.7,435.7 L570.2,435.7 L575.8,435.7 L581.3,435.7 L586.8,435.7 L592.3,435.7 L597.9,435.7 L603.4,435.7
 L608.9,435.7 L614.4,435.7 L619.9,435.7 L625.5,435.7 L631.0,435.7 L636.5,435.7 L642.0,435.7 L647.6,435.7
 L653.1,435.7 L658.6,435.7 L664.1,435.7 L669.6,435.7 L675.2,467.6 L680.7,467.7 L686.2,470.8 L691.7,470.8
 L697.3,470.9 L702.8,470.9 L708.3,470.9 L713.8,470.9 L719.3,470.9 L724.9,470.9 L730.4,470.9 L735.9,470.9
 L741.4,470.9 L747.0,470.9 L752.5,470.9 L758.0,470.9 L763.5,470.9 L769.0,470.9 L774.6,470.9 L780.1,470.9
 L785.6,470.9 L791.1,470.9 L796.7,470.9 L802.2,470.9 L807.7,470.9 L813.2,470.9 L818.8,470.9 L824.3,470.9
 L829.8,470.9 L835.3,470.9 L840.8,470.9 L846.4,470.9 L851.9,470.9 L857.4,470.9 L862.9,470.9 L868.5,470.9
 L874.0,470.9 L879.5,470.9 L885.0,470.9 L890.5,470.9 L896.1,470.9 L901.6,470.9 L907.1,470.9 L912.6,470.9
 L918.2,470.9 L923.7,470.9 L929.2,470.9 L934.7,470.9 L940.2,495.9 L945.8,495.7 L951.3,495.7 L956.8,495.8
 L962.3,495.8 L967.9,495.8 L973.4,495.8 L978.9,495.8 L984.4,495.8 L989.9,495.8 L995.5,495.8 L1001.0,495.8
 L1006.5,495.8 L1012.0,495.8 L1017.6,495.8 L1023.1,495.8 L1028.6,495.8 L1034.1,495.8 L1039.7,495.8 L1045.2,495.8
 L1050.7,495.8 L1056.2,495.8 L1061.7,495.8 L1067.3,495.8 L1072.8,495.8 L1078.3,495.8 L1083.8,495.8 L1089.4,495.8
 L1094.9,495.8 L1100.4,495.8 L1105.9,495.8 L1111.4,495.8 L1117.0,495.8 L1122.5,495.8 L1128.0,495.8 L1133.5,495.8
 L1139.1,495.8 L1144.6,495.8 L1150.1,495.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iF

## Residual History

In [68]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M11.3,57.1 L64.7,57.1 M45.6,83.2 L49.1,85.8 L52.6,86.8 L56.2,88.3 L59.7,96.7 L63.2,107.4
 L66.7,116.8 L70.3,118.3 L73.8,120.2 L77.3,124.1 L80.8,130.0 L84.3,147.7 L87.9,171.6 L91.4,181.3
 L94.9,171.9 L98.4,194.4 L102.0,204.1 L105.5,216.9 L109.0,220.1 L112.5,223.7 L116.1,230.3 L119.6,232.6
 L123.1,235.4 L126.6,250.1 L130.1,250.5 L133.7,250.5 L137.2,251.8 L140.7,252.2 L144.2,252.6 L147.8,253.0
 L151.3,253.7 L154.8,254.2 L158.3,255.2 L161.8,256.9 L165.4,259.2 L168.9,259.4 L172.4,260.8 L175.9,261.1
 L179.5,261.1 L183.0,261.2 L186.5,261.6 L190.0,262.4 L193.5,262.8 L197.1,262.9 L200.6,262.9 L204.1,263.0
 L207.6,263.0 L211.2,263.0 L214.7,263.0 L218.2,263.0 L221.7,263.0 L225.2,90.8 L228.8,101.1 L232.3,107.6
 L235.8,111.6 L239.3,119.1 L242.9,128.6 L246.4,137.4 L249.9,151.0 L253.4,139.2 L257.0,157.7 L260.5,234.3
 L264.0,258.9 L267.5,264.9 L271.0,274.7 L274.6,277.5 L278.1,280.2 L281.6,286.4 L285.1,288.0 L288.7,291.5
 L292.2,292.7 L295.7,293.3 L299.2,294.5 L302.7,295.9 L306.3,297.1 L309.8,297.4 L313.3,297.7 L316.8,298.8
 L320.4,300.5 L323.9,300.6 L327.4,300.8 L330.9,301.4 L334.4,302.0 L338.0,302.3 L341.5,302.8 L345.0,303.6
 L348.5,303.9 L352.1,304.3 L355.6,304.4 L359.1,304.9 L362.6,305.0 L366.1,305.1 L369.7,305.1 L373.2,305.2
 L376.7,305.3 L380.2,305.4 L383.8,305.5 L387.3,305.5 L390.8,305.6 L394.3,305.7 L397.9,305.7 L401.4,305.8
 L404.9,305.8 L408.4,305.8 L411.9,305.9 L415.5,305.9 L419.0,306.0 L422.5,306.0 L426.0,306.1 L429.6,306.1
 L433.1,306.1 L436.6,306.2 L440.1,306.2 L443.6,306.2 L447.2,161.6 L450.7,228.3 L454.2,266.5 L457.7,284.7
 L461.3,298.0 L464.8,304.5 L468.3,326.7 L471.8,326.8 L475.3,328.7 L478.9,333.5 L482.4,337.1 L485.9,340.5
 L489.4,345.3 L493.0,347.0 L496.5,347.1 L500.0,347.2 L503.5,347.4 L507.0,347.4 L510.6,348.8 L514.1,351.0
 L517.6,352.2 L521.1,353.2 L524.7,354.1 L528.2,355.3 L531.7,355.4 L535.2,356.0 L538.8,356.0 L542.3,356.1
 L545.8,356.6 L549.3,357.2 L552.8,357.2 L556.4,357.3 L559.9,357.4 L563.4,357.6 L566.9,357.7 L570.5,357.8
 L574.0,357.9 L577.5,357.9 L581.0,358.0 L584.5,358.0 L588.1,358.1 L591.6,358.2 L595.1,358.2 L598.6,358.2
 L602.2,358.3 L605.7,358.3 L609.2,358.3 L612.7,358.3 L616.2,278.7 L619.8,299.1 L623.3,316.0 L626.8,325.0
 L630.3,347.3 L633.9,348.4 L637.4,350.9 L640.9,354.1 L644.4,354.7 L647.9,359.7 L651.5,361.5 L655.0,364.9
 L658.5,368.4 L662.0,376.3 L665.6,376.4 L669.1,382.3 L672.6,382.7 L676.1,384.4 L679.7,385.7 L683.2,386.3
 L686.7,391.5 L690.2,391.7 L693.7,391.8 L697.3,391.8 L700.8,392.5 L704.3,392.6 L707.8,392.6 L711.4,392.6
 L714.9,393.3 L718.4,393.3 L721.9,393.7 L725.4,393.8 L729.0,393.9 L732.5,394.0 L736.0,394.0 L739.5,394.0
 L743.1,394.0 L746.6,394.0 L750.1,394.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M11.3,81.1 L64.7,81.1 

save results to text file

In [69]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [70]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:18:57 PM	185058d1...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:21:15 PM	704ed890...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/31/2024 9:22:09 PM	f868f5b3...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	3/31/2024 9:27:03 PM	625022d8...


Number of cells

In [71]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [72]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(4,ses.Count());


Function used to calculate the EOCs

In [73]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

In [74]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.WriteLine($"P={p},sess={iSess}");
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
        }
    }
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

In [75]:
EOCs

[ 0.9013796008869436, 1.9177500743146874, 2.3718684309422873, 2.578680651549844 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [76]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlux0"));
sessions

In [77]:
var si = sessions.Pick(0);
si

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'index')
   at System.Linq.ThrowHelper.ThrowArgumentOutOfRangeException(ExceptionArgument argument)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at Submission#78.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

plot it

In [78]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#79.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)